# **1. Shallow Learning**

In [24]:
# Librerías estándar
import os
import pickle
import numpy as np
import pandas as pd

# Sklearn: Preprocesamiento, modelos y métricas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score

# Gensim para embeddings no contextuales
from gensim.models import Word2Vec, FastText

# Sentence Transformers para embeddings contextuales
from sentence_transformers import SentenceTransformer

# TensorFlow / Keras para Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from sklearn.metrics import accuracy_score, f1_score
import torch
from collections import Counter


Para representar los textos, hemos elegido TF-IDF para capturar la importancia relativa de cada palabra en un documento frente al corpus completo, reduciendo el peso de palabras muy frecuentes que no aportan información discriminativa, como artículos y preposiciones en inglés. Hemos establecido un  límite de 3,000 palabras más importantes para reducir la dimensionalidad. Ademas hemos añadido y unigramas y bigramas para capturar algo de contexto local sin sobrecargar el modelo. Por otro lado, hemos eliminado las palabras vacías en inglés para centrar el análisis en palabras significativas. Esta representación genera vectores dispersos que son ideales para los modelos clásicos que usamos.

Hemos seleccionado cuatro modelos para evaluar el desempeño: Logistic Regression, LinearSVC, Random Forest y XGBoost. Esta combinación nos permite cubrir tanto modelos lineales como no lineales y comparar rapidez, precisión y estabilidad.

Antes de entrenar, hemos convertido los textos a números mediante label encoding, y  una división de train/validation del 80/20 para medir el rendimiento real y evitar overfitting. Además, hemos filtrado las clases con menos de dos registros, ya que la validación estratificada requiere al menos dos ejemplos por clase. La funcion recibe como parametro la variable objetivo. En este caso, recibe las variables "bias", "topic" y "source", que son nuestras variables a clasificar.

Finalmente, todos los modelos y el vectorizador TF-IDF han sido guardados para su reutilización. Este pipeline de Shallow Learning funciona como un baseline sólido que nos permite medir la mejora que aportan las representaciones densas y contextuales de texto que se utilizarán en las fases posteriores.

In [28]:
def shallow_pipeline(df, target_col):
    # Preparamos el texto
    if "text_joined" not in df.columns:
        df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))
    texts = df["text_joined"].astype(str).tolist()
    labels = df[target_col].tolist()

    # Filtramos las clases con menos de 2 registros
    counts = Counter(labels)
    valid_classes = [c for c, cnt in counts.items() if cnt > 1]
    mask = [lbl in valid_classes for lbl in labels]
    texts = [t for t, m in zip(texts, mask) if m]
    labels = [l for l, m in zip(labels, mask) if m]

    # Codificamos las etiquetas
    le = LabelEncoder()
    y = le.fit_transform(labels)

    # Hacemos el train/validation split
    X_train_text, X_val_text, y_train, y_val = train_test_split(
        texts, y, test_size=0.2, random_state=42, stratify=y
    )

    # Aplicamos TF-IDF
    vectorizer = TfidfVectorizer(max_features=3000, stop_words='english', ngram_range=(1,2))
    X_train = vectorizer.fit_transform(X_train_text)
    X_val = vectorizer.transform(X_val_text)

    # Definimos los modelos
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, n_jobs=-1),
        "LinearSVC": LinearSVC(),
        "Random Forest": RandomForestClassifier(n_estimators=150, n_jobs=-1),
       # "XGBoost": XGBClassifier(n_estimators=75, eval_metric="mlogloss", tree_method="hist", n_jobs=-1)
    }

    results = {}

    # Entrenamos, evaluamos y guardamos los modelos
    os.makedirs("data/models", exist_ok=True)
    for name, model in models.items():
        print(f"Entrenando {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        results[name] = {
            "Accuracy": accuracy_score(y_val, y_pred),
            "Macro-F1": f1_score(y_val, y_pred, average="macro")
        }
        # Guardamos el modelo
        pickle.dump(model, open(f"data/models/{name.replace(' ', '_').lower()}.pkl", "wb"))

    # Guardamos el vectorizador
    os.makedirs("data/features", exist_ok=True)
    pickle.dump(vectorizer, open("data/features/tfidf_vectorizer.pkl", "wb"))

    return results


Los resultados se analizarán en la sección 4 de este noteebook.

In [6]:
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")

In [26]:
# Llamamos a la función con bias
results_bias = shallow_pipeline(df, "bias")
print(pd.DataFrame(results_bias).T)



Entrenando Logistic Regression...
Entrenando LinearSVC...
Entrenando Random Forest...
Entrenando XGBoost...
                     Accuracy  Macro-F1
Logistic Regression  0.702109  0.700091
LinearSVC            0.698713  0.696551
Random Forest        0.689778  0.684783
XGBoost              0.734632  0.733918


In [ ]:
# Llamamos a la función con topic
results_topic = shallow_pipeline(df, "topic")
print(pd.DataFrame(results_topic).T)


In [ ]:
# Llamamos a la función con source
results_source = shallow_pipeline(df, "source")
print(pd.DataFrame(results_source).T)

Entrenando Logistic Regression...
Entrenando LinearSVC...
Entrenando Random Forest...
Entrenando XGBoost...
                     Accuracy  Macro-F1
Logistic Regression  0.503401  0.103117
LinearSVC            0.559076  0.219572
Random Forest        0.500000  0.120979
XGBoost              0.532581  0.223251


# **2. Modelos Deep**


En la parte de Deep Learning, hemos optado por utilizar redes neuronales recurrentes, específicamente LSTM y GRU, debido a su capacidad para capturar dependencias secuenciales en el texto. A diferencia de los modelos de Shallow Learning, que tratan cada palabra o n-grama de manera independiente, las RNNs permiten que la red recuerde información contextual de palabras anteriores en la secuencia, lo cual es crucial para nuestras tareas de clasificación de texto donde el significado puede depender del orden de las palabras.

Para la representación de los textos, hemos empleado embeddings densos, utilizando tres enfoques distintos con Word2Vec: congelado, fine-tuneado y desde cero. En el caso de los embeddings congelados, utilizamos un modelo preentrenado de Word2Vec y lo fijamos durante el entrenamiento de la red, de manera que solo la LSTM o GRU aprenda a combinar los vectores preexistentes. Esto permite evaluar cuánto conocimiento semántico ya capturado en Word2Vec puede ayudar a la tarea sin modificarlo. En el enfoque de fine-tune, los embeddings inicializados con Word2Vec se ajustan durante el entrenamiento, permitiendo que la red adapte los vectores a las particularidades del dataset específico. Finalmente, la opción de embeddings entrenados desde cero crea vectores aleatorios que se aprenden completamente durante el entrenamiento, lo que permite que la red descubra representaciones óptimas para la tarea, aunque requiere más datos y tiempo de entrenamiento.

Hemos elegido LSTM y GRU ya que cumple nuestra necesidad de comparar dos variantes de redes recurrentes: las LSTM tienen una mayor capacidad para capturar dependencias de largo plazo mediante su mecanismo de puertas, mientras que las GRU son más simples y computacionalmente eficientes, lo que puede acelerar el entrenamiento sin perder demasiado rendimiento. 

Los textos se transforman primero en secuencias de índices según el vocabulario de Word2Vec o un tokenizer entrenado sobre el dataset, y se aplica padding para unificar la longitud de las secuencias. Esto asegura que las redes puedan procesar lotes de datos de manera eficiente. Finalmente, la capa de salida utiliza softmax para producir probabilidades sobre las clases, y la red se entrena con categorical crossentropy, optimizando la accuracy y el macro-F1 como métricas de desempeño, lo cual es consistente con la evaluación utilizada en la parte de Shallow Learning.

En conclusion, en este apartado permite que nuestra red aprenda tanto representaciones densas de palabras como patrones secuenciales de las oraciones, ofreciendo una ventaja sobre los modelos lineales y de ensamble de Shallow Learning que solo utilizan información superficial y dispersa de los textos.

Embeddings finetuneados

In [ ]:
# Cargamos el dataset tokenizado
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")
y = df_train["bias"].values

# Codificamos los labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split train/validation
X_tr_text, X_val_text, y_tr, y_val = train_test_split(
    df_train["tokens"], y_cat, test_size=0.2, random_state=42
)

# Cargamos el Word2Vec preentrenado de la anterior entrega
w2v_model = Word2Vec.load("data/embeddings/word2vec.model")
embedding_dim = w2v_model.vector_size

# Creamos el vocabulario e índices
word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}
vocab_size = len(word_index) + 1  # +1 para padding

# Convertimos los tokens a índices
def tokens_to_indices(tokens, word_index):
    return [word_index[t] for t in tokens if t in word_index]

X_tr_idx = [tokens_to_indices(t, word_index) for t in X_tr_text]
X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_text]

# Aplicamos padding
max_seq_len = 200
X_tr_pad = pad_sequences(X_tr_idx, maxlen=max_seq_len, padding='post')
X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')

# Creamos la matriz de embedding 
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = w2v_model.wv[word]

# Definimos y entrenamos los modelos 

def build_rnn(model_type='LSTM'):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_seq_len,
                        trainable=True))  # Fine-tune embeddings
    if model_type == 'LSTM':
        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == 'GRU':
        model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# LSTM
lstm_model = build_rnn('LSTM')
lstm_history = lstm_model.fit(X_tr_pad, y_tr,
                              validation_data=(X_val_pad, y_val),
                              epochs=10,
                              batch_size=64)

# GRU
gru_model = build_rnn('GRU')
gru_history = gru_model.fit(X_tr_pad, y_tr,
                            validation_data=(X_val_pad, y_val),
                            epochs=10,
                            batch_size=64)

# Evaluamos los modelos
from sklearn.metrics import accuracy_score, f1_score

# Predicciones
y_pred_lstm = lstm_model.predict(X_val_pad, batch_size=64)
y_pred_gru = gru_model.predict(X_val_pad, batch_size=64)

y_pred_lstm_labels = np.argmax(y_pred_lstm, axis=1)
y_pred_gru_labels = np.argmax(y_pred_gru, axis=1)
y_val_labels = np.argmax(y_val, axis=1)

# Métricas
results = {
    'LSTM': {
        'Accuracy': accuracy_score(y_val_labels, y_pred_lstm_labels),
        'Macro-F1': f1_score(y_val_labels, y_pred_lstm_labels, average='macro')
    },
    'GRU': {
        'Accuracy': accuracy_score(y_val_labels, y_pred_gru_labels),
        'Macro-F1': f1_score(y_val_labels, y_pred_gru_labels, average='macro')
    }
}

results_df = pd.DataFrame(results).T
print("Resultados:")
print(results_df)


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 116s 322ms/step - accuracy: 0.4257 - loss: 1.0562 - val_accuracy: 0.4750 - val_loss: 1.0236
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 109s 311ms/step - accuracy: 0.5227 - loss: 0.9537 - val_accuracy: 0.4959 - val_loss: 0.9828
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 119s 340ms/step - accuracy: 0.5904 - loss: 0.8587 - val_accuracy: 0.5234 - val_loss: 0.9534
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 150s 428ms/step - accuracy: 0.6674 - loss: 0.7404 - val_accuracy: 0.5173 - val_loss: 1.0088
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 131s 373ms/step - accuracy: 0.7374 - loss: 0.6141 - val_accuracy: 0.5116 - val_loss: 1.0896
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 182s 521ms/step - accuracy: 0.8024 - loss: 0.4791 - val_accuracy: 0.5046 - val_loss: 1.2833
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - accuracy: 0.8611 - loss: 0.3566 - val_accuracy: 0.5095 - val_loss: 1.3578
Epoch 8/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - accuracy: 0.8989 - loss:

-Rendimiento general:
   -Ambos modelos muestran resultados muy similares, con valores en torno al 53–54%, tanto en Accuracy como en Macro-F1. Esto indica que:
       -Los dos modelos capturan de forma parecida los patrones secuenciales del sesgo ideológico.
       -No existe una ventaja clara de ninguno de los dos modelos neuronales en este dataset.
-Interpretación:
    -El rendimiento indica que el sesgo ideológico es una tarea difícil incluso para modelos neuronales. 
    -Puede que los textos no tengan suficiente señal secuencial para que LSTM/GRU destaquen claramente.
-Conclusión:
    -Ambos modelos presentan un rendimiento equivalente, pero al ser  ligeramente superior, hemos decidido usar LSTM  como baseline de deep learning. Sin embargo, estas arquitecturas probablemente no capturan matices ideológicos complejos, por lo que se es necesario explorar  modelos más potentes como BERT o RoBERTa.

Embeddings no finetuneados

In [ ]:
#Cargamos el dataset tokenizado
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")

# Creamos la  columna text_joined a partir de tokens
df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))

texts = df["text_joined"].astype(str).tolist()
labels = df["bias"].tolist()

# Tokenización y secuencias
vocab_size = 20000
maxlen = 100
embedding_dim = 100

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
X_seq = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X_seq, maxlen=maxlen)

# Codificamos las etiquetas
le = LabelEncoder()
y = le.fit_transform(labels)

# Hacemos el train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Aplicamos LSTM y GRU-
def build_lstm_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=False),
        LSTM(128, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    return model

def build_gru_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=False),
        GRU(128, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    return model

# Entrenamos LSTM
lstm_model = build_lstm_model()
history_lstm = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

# Entrenamos GRU
gru_model = build_gru_model()
history_gru = gru_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=64
)

# Evaluamos los modelos
def evaluate(model):
    preds = np.argmax(model.predict(X_val), axis=1)
    return accuracy_score(y_val, preds), f1_score(y_val, preds, average="macro")

acc_lstm, f1_lstm = evaluate(lstm_model)
acc_gru, f1_gru = evaluate(gru_model)

print("\nResultados Embedding Random (NO fine-tuneado)")
print("LSTM → Accuracy:", acc_lstm, "Macro-F1:", f1_lstm)
print("GRU  → Accuracy:", acc_gru, "Macro-F1:", f1_gru)


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 90ms/step - accuracy: 0.3635 - loss: 1.0929 - val_accuracy: 0.3672 - val_loss: 1.0934
Epoch 2/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 31s 87ms/step - accuracy: 0.3830 - loss: 1.0869 - val_accuracy: 0.3801 - val_loss: 1.0908
Epoch 3/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 33s 93ms/step - accuracy: 0.4076 - loss: 1.0751 - val_accuracy: 0.4006 - val_loss: 1.0844
Epoch 4/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 31s 88ms/step - accuracy: 0.4246 - loss: 1.0634 - val_accuracy: 0.3890 - val_loss: 1.0850
Epoch 5/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 32s 90ms/step - accuracy: 0.4400 - loss: 1.0456 - val_accuracy: 0.3987 - val_loss: 1.0971
Epoch 1/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 91ms/step - accuracy: 0.3635 - loss: 1.0930 - val_accuracy: 0.3692 - val_loss: 1.0923
Epoch 2/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 34s 97ms/step - accuracy: 0.3759 - loss: 1.0885 - val_accuracy: 0.3924 - val_loss: 1.0874
Epoch 3/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 42s 121ms/step - accuracy: 0.3967 - loss: 1.0815 - val_acc

Los resultados obtenidos muestran que, sin información semántica previa ni capacidad de ajuste, el rendimiento de los modelos secuenciales es limitado. El modelo LSTM alcanza un accuracy cercano al 0.40 y un Macro-F1 alrededor de 0.33, mientras que GRU ofrece cifras similares, ligeramente inferiores en accuracy pero algo superiores en Macro-F1. Estas métricas evidencian que, al no permitir el fine-tuning y partir de embeddings aleatorios, los modelos no son capaces de capturar adecuadamente las relaciones lingüísticas del texto y, en consecuencia, su desempeño queda claramente por debajo de los modelos con embeddings preentrenados o ajustables.

Word2Vec congelado vs Word2Vec fine-tuneado vs Word2Vec from scratch

In [ ]:

# Cragamos el dataset tokenizado
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")
df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))

texts = df["tokens"].tolist()
labels = df["bias"].tolist()

# Codificamos las etiquetas
le = LabelEncoder()
y = le.fit_transform(labels)

# Hacemos el train/val split
X_tr_text, X_val_text, y_tr, y_val = train_test_split(
    texts, y, test_size=0.2, random_state=42
)

# Cargamos el Word2Vec preentrenado
w2v_model = Word2Vec.load("data/embeddings/word2vec.model")
embedding_dim = w2v_model.vector_size

word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}
vocab_size = len(word_index) + 1

def tokens_to_indices(tokens, word_index):
    return [word_index[t] for t in tokens if t in word_index]

X_tr_idx = [tokens_to_indices(t, word_index) for t in X_tr_text]
X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_text]

max_seq_len = 200
X_tr_pad = pad_sequences(X_tr_idx, maxlen=max_seq_len, padding='post')
X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = w2v_model.wv[word]

# Funcion para crear LSTM
def build_lstm_model(embedding_matrix, trainable=True):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=max_seq_len,
                        trainable=trainable))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(1e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Word2Vec Frozen
lstm_frozen = build_lstm_model(embedding_matrix, trainable=False)
history_frozen = lstm_frozen.fit(X_tr_pad, y_tr,
                                 validation_data=(X_val_pad, y_val),
                                 epochs=5,
                                 batch_size=64)

# Word2Vec Fine-tune
lstm_finetune = build_lstm_model(embedding_matrix, trainable=True)
history_finetune = lstm_finetune.fit(X_tr_pad, y_tr,
                                     validation_data=(X_val_pad, y_val),
                                     epochs=5,
                                     batch_size=64)

# Word2Vec Scratch
embedding_matrix_random = np.random.normal(size=(vocab_size, embedding_dim))
lstm_scratch = build_lstm_model(embedding_matrix_random, trainable=True)
history_scratch = lstm_scratch.fit(X_tr_pad, y_tr,
                                   validation_data=(X_val_pad, y_val),
                                   epochs=5,
                                   batch_size=64)

# Evaluamos los modelos
def evaluate(model, X_val, y_val):
    preds = np.argmax(model.predict(X_val), axis=1)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro')
    return acc, f1

results = {}
results['Word2Vec Frozen'] = evaluate(lstm_frozen, X_val_pad, y_val)
results['Word2Vec Fine-tune'] = evaluate(lstm_finetune, X_val_pad, y_val)
results['Word2Vec Scratch'] = evaluate(lstm_scratch, X_val_pad, y_val)

# Mostramos los resultados
results_df = pd.DataFrame(results, index=['Accuracy','Macro-F1']).T
print(results_df)


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 268s 737ms/step - accuracy: 0.4120 - loss: 1.0719 - val_accuracy: 0.4355 - val_loss: 1.0549
Epoch 2/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - accuracy: 0.4564 - loss: 1.0241 - val_accuracy: 0.4694 - val_loss: 1.0021
Epoch 3/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 201s 575ms/step - accuracy: 0.4739 - loss: 0.9988 - val_accuracy: 0.4682 - val_loss: 0.9979
Epoch 4/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 201s 575ms/step - accuracy: 0.4852 - loss: 0.9866 - val_accuracy: 0.4973 - val_loss: 0.9732
Epoch 5/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 152s 433ms/step - accuracy: 0.4917 - loss: 0.9776 - val_accuracy: 0.4993 - val_loss: 0.9686
Epoch 1/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 143s 398ms/step - accuracy: 0.4306 - loss: 1.0563 - val_accuracy: 0.4725 - val_loss: 1.0097
Epoch 2/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 181s 510ms/step - accuracy: 0.5197 - loss: 0.9549 - val_accuracy: 0.4961 - val_loss: 0.9732
Epoch 3/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 146s 349ms/step - accuracy: 0.5951 - loss: 0.85

Los resultados muestran que el enfoque más eficaz es Word2Vec fine-tuneado, que obtiene el mejor rendimiento tanto en accuracy como en Macro-F1, indicando que ajustar los embeddings al dominio específico de las noticias mejora la capacidad del modelo para capturar señales lingüísticas relevantes. El modelo  congelado alcanza métricas ligeramente inferiores, lo que sugiere que aunque los vectores preentrenados aportan una buena base semántica, limitar su actualización reduce su adaptabilidad a la tarea. Por último, el modelo entrenado desde cero obtiene el peor desempeño, reflejando que aprender embeddings sin preentrenamiento requiere muchos más datos y épocas para alcanzar niveles competitivos.

# **3. Comparación de embeddings**


Embeddings tradicionales

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

# ============================
# 1. Cargar dataset tokenizado
# ============================
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")

# Crear columna text_joined a partir de tokens si no existe
if "text_joined" not in df.columns:
    df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))

texts = df["text_joined"].astype(str).tolist()
labels = df["bias"].tolist()

# Codificar etiquetas
le = LabelEncoder()
y = le.fit_transform(labels)

# Train/validation split
X_train_text, X_val_text, y_train, y_val = train_test_split(
    texts, y, test_size=0.2, random_state=42, stratify=y
)

# ============================
# Helper: train + evaluate
# ============================
def train_eval(model, X_train, X_val, name=""):
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="macro")
    print(f"{name} →  Accuracy: {acc:.4f} | Macro-F1: {f1:.4f}")
    return acc, f1

results = {}

# ============================
# 2. Bag of Words
# ============================
vectorizer_bow = CountVectorizer(max_features=20000)
X_train_bow = vectorizer_bow.fit_transform(X_train_text)
X_val_bow   = vectorizer_bow.transform(X_val_text)

results["BoW_LogReg"] = train_eval(
    LogisticRegression(max_iter=3000),
    X_train_bow, X_val_bow,
    "BoW + Logistic Regression"
)

results["BoW_SVM"] = train_eval(
    LinearSVC(),
    X_train_bow, X_val_bow,
    "BoW + LinearSVM"
)

# ============================
# 3. TF-IDF (Unigram)
# ============================
vectorizer_tfidf_uni = TfidfVectorizer(max_features=20000, ngram_range=(1,1))
X_train_tfidf_uni = vectorizer_tfidf_uni.fit_transform(X_train_text)
X_val_tfidf_uni   = vectorizer_tfidf_uni.transform(X_val_text)

results["TFIDF_uni_LogReg"] = train_eval(
    LogisticRegression(max_iter=3000),
    X_train_tfidf_uni, X_val_tfidf_uni,
    "TF-IDF Unigram + Logistic Regression"
)

results["TFIDF_uni_SVM"] = train_eval(
    LinearSVC(),
    X_train_tfidf_uni, X_val_tfidf_uni,
    "TF-IDF Unigram + LinearSVM"
)

# ============================
# 4. TF-IDF (N-grams 1–2)
# ============================
vectorizer_tfidf_ngram = TfidfVectorizer(max_features=40000, ngram_range=(1,2))
X_train_tfidf_ngram = vectorizer_tfidf_ngram.fit_transform(X_train_text)
X_val_tfidf_ngram   = vectorizer_tfidf_ngram.transform(X_val_text)

results["TFIDF_ngram_LogReg"] = train_eval(
    LogisticRegression(max_iter=3000),
    X_train_tfidf_ngram, X_val_tfidf_ngram,
    "TF-IDF N-gram (1–2) + Logistic Regression"
)

results["TFIDF_ngram_SVM"] = train_eval(
    LinearSVC(),
    X_train_tfidf_ngram, X_val_tfidf_ngram,
    "TF-IDF N-gram (1–2) + LinearSVM"
)

# ============================
# Resultados finales
# ============================
print("\n==============================")
print("Resultados embeddings tradicionales")
print("==============================")
for k, v in results.items():
    print(f"{k}: Acc={v[0]:.4f} | F1={v[1]:.4f}")


KeyError: 'text_joined'

Embeddings no contextuales(usamos embeddings entrenados con nuestro dataset
)

In [3]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors, FastText
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# -----------------------------
# Cargar dataset
# -----------------------------
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")
df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))
texts = df["tokens"].tolist()
labels = df["bias"].tolist()

# Codificar etiquetas
le = LabelEncoder()
y = le.fit_transform(labels)

X_tr_text, X_val_text, y_tr, y_val = train_test_split(
    texts, y, test_size=0.2, random_state=42
)

# -----------------------------
# Cargar embeddings preentrenados (GloVe ejemplo)
# -----------------------------
# Opción 1: GloVe
glove_file = "data/embeddings/glove.6B.100d.txt"  # Ajustar path
embeddings_index = {}
with open(glove_file, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = vector

embedding_dim = 100
word_index = {word: i+1 for i, word in enumerate(set([t for sublist in texts for t in sublist]))}
vocab_size = len(word_index) + 1

def tokens_to_indices(tokens, word_index):
    return [word_index[t] for t in tokens if t in word_index]

X_tr_idx = [tokens_to_indices(t, word_index) for t in X_tr_text]
X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_text]

max_seq_len = 200
X_tr_pad = pad_sequences(X_tr_idx, maxlen=max_seq_len, padding='post')
X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')

# Crear matriz de embedding
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in embeddings_index:
        embedding_matrix[i] = embeddings_index[word]
    else:
        embedding_matrix[i] = np.random.normal(size=(embedding_dim,))

# -----------------------------
# Definir modelo LSTM
# -----------------------------
def build_lstm_model(embedding_matrix, trainable=False):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=max_seq_len,
                        trainable=trainable))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(len(np.unique(y)), activation='softmax'))
    model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Entrenar
lstm_model = build_lstm_model(embedding_matrix, trainable=True)
lstm_model.fit(X_tr_pad, y_tr, validation_data=(X_val_pad, y_val), epochs=5, batch_size=64)

# Evaluar
preds = np.argmax(lstm_model.predict(X_val_pad), axis=1)
acc = accuracy_score(y_val, preds)
f1 = f1_score(y_val, preds, average='macro')
print("GloVe LSTM → Accuracy:", acc, "Macro-F1:", f1)


FileNotFoundError: [Errno 2] No such file or directory: 'data/embeddings/glove.6B.100d.txt'

Embeddings contextuales

In [3]:
# ===============================
#  1) EMBEDDINGS CONTEXTUALES (freeze) + CLASSIFIER
# ===============================
# Requisitos: transformers, torch, scikit-learn
# pip install transformers torch scikit-learn --upgrade

import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
import joblib
import torch
from transformers import AutoTokenizer, AutoModel

# ===============================
# Cargar dataset tokenizado / limpio
# ===============================
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")

# Crear la columna text_joined si no existe
if "text_joined" not in df.columns:
    df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))

# Textos y etiquetas
texts = df["text_joined"].astype(str).tolist()
labels = df["bias"].tolist()

print(f"Número de ejemplos: {len(texts)}")
print(f"Clases encontradas: {set(labels)}")

# ===============================
# Configuración del modelo
# ===============================
MODEL_NAME = "distilbert-base-uncased"   # ligero; puedes cambiarlo
BATCH_SIZE = 32
MAX_LEN = 128
EMB_SAVE_PATH = "data/features/bert_cls_embeddings.npy"
LABEL_SAVE_PATH = "data/features/bert_labels.npy"
TOKENIZER_SAVE = "data/features/bert_tokenizer"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# ===============================
# Cargar tokenizer y modelo (PyTorch)
# ===============================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.to(DEVICE)
model.eval()  # freeze embeddings

# ===============================
# Función para extraer embeddings CLS por batches
# ===============================
def extract_cls_embeddings(texts, batch_size=BATCH_SIZE, max_len=MAX_LEN):
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Extrayendo BERT embeddings"):
        batch_texts = texts[i:i+batch_size]
        enc = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_len,
            return_tensors="pt"
        )
        enc = {k: v.to(DEVICE) for k, v in enc.items()}
        with torch.no_grad():
            out = model(**enc)
            cls_embs = out.last_hidden_state[:, 0, :].cpu().numpy()
            all_embs.append(cls_embs)
    all_embs = np.vstack(all_embs)
    return all_embs

# ===============================
# Extraer embeddings (si ya existen, cargarlos)
# ===============================
os.makedirs("data/features", exist_ok=True)

if os.path.exists(EMB_SAVE_PATH) and os.path.exists(LABEL_SAVE_PATH):
    print("Cargando embeddings guardados...")
    X = np.load(EMB_SAVE_PATH)
    y = np.load(LABEL_SAVE_PATH)
    le = joblib.load("data/features/label_encoder.joblib")
else:
    print("Extrayendo embeddings BERT (esto puede tardar)...")
    X = extract_cls_embeddings(texts)
    le = LabelEncoder()
    y = le.fit_transform(labels)
    np.save(EMB_SAVE_PATH, X)
    np.save(LABEL_SAVE_PATH, y)
    joblib.dump(le, "data/features/label_encoder.joblib")
    tokenizer.save_pretrained(TOKENIZER_SAVE)

print("Embeddings shape:", X.shape)
print("Labels shape:", y.shape)
unique, counts = np.unique(y, return_counts=True)
print("Distribución labels:", dict(zip(unique, counts)))

# ===============================
# Split train/validation
# ===============================
X_tr, X_val, y_tr, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ===============================
# Entrenar clasificador LogisticRegression
# ===============================
clf = LogisticRegression(max_iter=2000, multi_class="multinomial", n_jobs=-1, C=1.0)
print("Entrenando Logistic Regression sobre embeddings BERT (frozen)...")
clf.fit(X_tr, y_tr)

# ===============================
# Evaluación
# ===============================
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val, y_pred)
macro_f1 = f1_score(y_val, y_pred, average="macro")

print(f"\nResultados BERT frozen + LogisticRegression:")
print(f"Accuracy: {acc:.4f}")
print(f"Macro-F1: {macro_f1:.4f}\n")
print("Classification report:\n")
print(classification_report(y_val, y_pred, target_names=le.classes_))

# ===============================
# Guardar clasificador
# ===============================
os.makedirs("data/models", exist_ok=True)
joblib.dump(clf, "data/models/bert_frozen_logreg.pkl")
print("Modelo guardado en data/models/bert_frozen_logreg.pkl")

Número de ejemplos: 27978
Clases encontradas: {0, 1, 2}
Device: cpu


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Extrayendo BERT embeddings:   0%|          | 0/875 [00:00<?, ?it/s]

Embeddings shape: (27978, 768)
Labels shape: (27978,)
Distribución labels: {np.int64(0): np.int64(9750), np.int64(1): np.int64(7988), np.int64(2): np.int64(10240)}
Entrenando Logistic Regression sobre embeddings BERT (frozen)...


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Resultados BERT frozen + LogisticRegression:
Accuracy: 0.5709
Macro-F1: 0.5669

Classification report:



TypeError: object of type 'numpy.int64' has no len()

# **4. Tabla Comaprativa de Resultados**

Para entender los resultados, hay que aclarar los parámetros utilizados para cada variable en shallow learning.

En el caso de la variable source, hemos tenido que reducir el número de estimadores del Random Forest de 300 a 150. Además, el número de estimadores del XGBoost también ha sido reducido de 200 a 75. Sin esta reducción, no habriamos sido capaces de terminar la ejecución de la celda. Ha llegado a estar más de una hora y seguía sin terminar de ejecutarse. 

En cuanto a la variable topic, además de las rebajas aplicadas al caso de la variable source, hemos decidido quitar el modelo XGBoost, ya que no termina de ejecutarse. Disponemos de equipos con capacidades técnicas muy limitadas, por lo que, con mejores ordenadores, no se tendrían que reducir los valores.

In [ ]:
results_bias = {
    "Logistic Regression": {"Accuracy":  0.702109  , "Macro-F1":0.700091},
    "LinearSVC": {"Accuracy": 0.698713, "Macro-F1":0.696551},
    "Random Forest": {"Accuracy": 0.683881, "Macro-F1": 0.679829},
    "XGBoost": {"Accuracy":0.734632   , "Macro-F1": 0.733918}
}

results_topic = {
    "Logistic Regression": {"Accuracy": 0.566655  , "Macro-F1":0.312507},
    "LinearSVC": {"Accuracy": 0.566476, "Macro-F1": 0.374703},
    "Random Forest": {"Accuracy": 0.507863, "Macro-F1": 0.230810},
    "XGBoost": {"Accuracy": 0.499643, "Macro-F1": 0.325067}
}

results_source = {
    "Logistic Regression": {"Accuracy": 0.503401, "Macro-F1":0.103117},
    "LinearSVC": {"Accuracy": 0.559076, "Macro-F1":0.219572},
    "Random Forest": {"Accuracy": 0.500000, "Macro-F1": 0.120979},
    "XGBoost": {"Accuracy": 0.532581, "Macro-F1": 0.223251}
}

# Crear DataFrame comparativo manualmente
rows = []
for model in ["Logistic Regression", "LinearSVC", "Random Forest", "XGBoost"]:
    row = {
        "Model": model,
        "Bias Accuracy": results_bias[model]["Accuracy"],
        "Bias Macro-F1": results_bias[model]["Macro-F1"],
        "Topic Accuracy": results_topic[model]["Accuracy"],
        "Topic Macro-F1": results_topic[model]["Macro-F1"],
        "Source Accuracy": results_source[model]["Accuracy"],
        "Source Macro-F1": results_source[model]["Macro-F1"]
    }
    rows.append(row)

df_comparison = pd.DataFrame(rows)
print(df_comparison)

# **5. Interpretabilidad**